### Import packages

In [1]:
import os
import glob
import numpy  as np
import pandas as pd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import ipywidgets as widgets
import soundfile as sf

### Choose file to inspect

In [2]:
directory='/Users/saroltagabulya/git/Orca/'
wavs=glob.glob('*.csv')
file=[]

dropdown=widgets.Dropdown(
    options=wavs,
    description='CSV files:',
    disabled=False,
)

def dropdown_eventhandler(change):
    file.clear()
    file.append(change.new)
    print(change.new)
dropdown.observe(dropdown_eventhandler, names='value')
display(dropdown)

Dropdown(description='CSV files:', options=('call_data_2017-02-04--10-14-06--00-16-35--BC_2.csv', 'time_stamps…

time_stamps_2017-02-04--10-14-06--00-16-35--BC_2.csv
time_stamps_2017-02-04--10-14-06--00-16-35--BC.csv


### Read csv file as pandas

In [3]:
filename = directory + file[0]
time_stamps=pd.read_csv(filename, index_col=0)

# Also extract recording name for later use
recording_name=file[0][12:-4]

#recording_name='2017-02-04--10-14-06--00-16-35--BC'

### Read metadata

In [4]:
try:
    wave_file=wave.open(file[0], "rb")
    sample_rate = wave_file.getframerate()
except:
    sample_rate=int(input('Please check sampling rate manually in the metadata file and set below in Hz! \n'))

Please check sampling rate manually in the metadata file and set below in Hz! 
100000


### Read in wav

In [5]:
y, sr = librosa.load(recording_name + '.wav', sr=sample_rate)

### Onsets & offsets in csv are in seconds => Convert to samples by multiplying them with the fs

second * fs = samples


In [6]:
time_stamps['onset_samples']=time_stamps.onset.apply(lambda x: int(round(x*sr)))
time_stamps['offset_samples']=time_stamps.offset.apply(lambda x: int(round(x*sr)))

### Function for cropping wav and saving

In [7]:
try:
    os.mkdir('./call_segments/' + recording_name)
    
except Exception as e:
    print(e)

[Errno 17] File exists: './call_segments/2017-02-04--10-14-06--00-16-35--BC'


In [18]:
def crop_call(row, y, sr, recording_name):
    call_filename= recording_name + '_' + str(row.onset_samples) + '_' + str(row.offset_samples) + '.wav'
    call=y[row.onset_samples: row.offset_samples]
    call_path='./call_segments/' + recording_name +'/' +  call_filename
    sf.write(call_path, call, sr) 
    return call_path

In [19]:
time_stamps['call_wav']=time_stamps.apply(lambda x: crop_call(x, y, sr, recording_name), axis=1)

### Create metadata files

### Save csv under new name

In [31]:
time_stamps.to_csv('call_data_' + recording_name + '_2'+ '.csv')